In [1]:
!pip install torch
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pandas as pd
import random

In [3]:
!gdown "1MUfxNB38jEojySdQmFTx854ClKfSL5VH&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1MUfxNB38jEojySdQmFTx854ClKfSL5VH&confirm=t
To: /content/Laifu_Mental.csv
100% 8.26M/8.26M [00:00<00:00, 33.5MB/s]


In [4]:
# Set the paths to your CSV files
csv_file = "/content/Laifu_Mental.csv"

# Set the desired ratio for the training set (e.g., 80%)
train_ratio = 0.8

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_file)

# Shuffle the DataFrame randomly
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the DataFrame into training and validation sets
split_index = int(len(df) * train_ratio)
train_df = df[:split_index]
val_df = df[split_index:]

# Save the training and validation sets to new CSV files
train_csv = "train.csv"
val_csv = "val.csv"
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Create TextDataset instances
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_csv,
    block_size=128
)

val_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=val_csv,
    block_size=128
)

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-fine-tuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
# Fine-tune the GPT-2 model
trainer.train()

Step,Training Loss
500,3.251900
1000,3.055000
1500,2.946300
2000,2.883100
2500,2.807800
3000,2.765700
3500,2.596200
4000,2.560200
4500,2.533000
5000,2.491700


TrainOutput(global_step=8853, training_loss=2.6123834271949655, metrics={'train_runtime': 1627.254, 'train_samples_per_second': 21.758, 'train_steps_per_second': 5.44, 'total_flos': 2312826421248000.0, 'train_loss': 2.6123834271949655, 'epoch': 3.0})

In [6]:
model.save_pretrained("/content/")
tokenizer.save_pretrained("/content/")

('/content/tokenizer_config.json',
 '/content/special_tokens_map.json',
 '/content/vocab.json',
 '/content/merges.txt',
 '/content/added_tokens.json')

In [7]:
# Set the model to evaluation mode
model.eval()

# Function to generate a response from the chatbot
def generate_response(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda:0')
    attention_mask = torch.ones(input_ids.shape, device='cuda:0')

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.5
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Test the model
while True:
    user_input = input("User: ")

    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break

    response = generate_response(user_input)
    print("Chatbot:", response)

User: i'm sad


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: i'm sad, but I can't help it. How do I get my life back?","Hello, and thank you for your question. \u00a0First and foremost, I want to commend you on reaching out to find a therapist who specializes in working with individuals who have experienced trauma and have a history of depression, anxiety, or any other issues that may be contributing to your current depression and anxiety. This is something that I am sure many of my clients struggle with and I would
User: i want to die


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: i want to die, but I don't know what to do.","I'm sorry to hear that you are struggling with this. \u00a0It sounds like you have a lot of unresolved issues that need to be worked through in order for you to move forward in your life and be happy and healthy. I would suggest talking with a local mental health professional to work through these issues and see if you can find a therapist that can help you with some of the issues you're grappling with.
User: i don't know waht to do


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: i don't know waht to do\u00a0anything by myself. I feel like I'm not worth knowing or \""not worth anything\"" to anyone.
   How can I get myself to just move on?","Hello, and thank you for your question. First, I want to say that I am very sorry that you are going through this. There are a lot of different things going on in your life right now. You may not know it, but
User: 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: ignored